# PWC - Quant Finance Modelling

##### Remark: 
The first part takes into account the 'real vintage' repayments that have already occurred and then computes the forecast based on that.

However, in the given solution, the forecast is computed for all repayment periods (perdiods 3 to 30 - so including those already paid) and then only the forecasted periods are used. For that jump in part II.

Take care the formula given is false! The right one is:
$$
p_i = \max \left\{ p_2 * \ln \left( 1 + \left( 1 - \left( 1 - \sum_{j=1}^{i-1} p_j \right) \right)*(1- \frac{i-1}{30}) \right); 0 \right\}
$$

## Part I
The first part takes into account the 'real vintage' repayments that have already occurred and then computes the forecast based on that.

In [1615]:
import pandas as pd
import numpy as np

In [1616]:
# To display all rows
pd.set_option('display.max_rows', None)

# To display all columns
pd.set_option('display.max_columns', None)

In [1617]:
df = pd.read_csv(r"C:\Users\lhoan\OneDrive\Bureau\Pwc - Forage\Data (2).csv",sep=';')
df = df.rename(columns={'Unnamed: 0': 'date'})

In [1618]:
df.head(100)
#df.tail()

,date,Origination Amount,31.05.2019,30.06.2019,31.07.2019,31.08.2019,30.09.2019,31.10.2019,30.11.2019,31.12.2019,31.01.2020,29.02.2020,31.03.2020,30.04.2020,31.05.2020,30.06.2020,31.07.2020,31.08.2020,30.09.2020,31.10.2020,30.11.2020,31.12.2020
0,31.05.2019,10018746.17,1443069.08,3332200.33,1328138.75,928085.74,736418.27,539403.31,427557.86,324459.32,237056.39,168364.60,116684.68,92699.67,63399.66,53265.12,37121.13,29787.10,24524.90,18085.94,16581.01,11442.97
1,30.06.2019,10868379.04,0.00,1392751.60,3011884.91,1237868.70,970929.28,892351.83,668767.02,505612.59,419598.74,329262.69,255222.42,198833.96,161996.73,138461.91,92346.68,79641.30,63457.44,52373.85,43374.70,37404.87
2,31.07.2019,10733932.61,0.00,0.00,1537650.24,2953335.55,1208316.08,879375.19,711016.84,658251.40,503465.03,423045.23,302575.54,258652.52,191798.05,170027.54,127574.33,110301.21,89766.69,64746.84,61408.92,50312.70
3,31.08.2019,12558727.02,0.00,0.00,0.00,1617681.94,4082016.00,1387474.94,1247623.59,886293.35,694348.63,571024.44,417223.56,336686.08,253556.20,200066.59,151859.74,109973.00,90228.14,70661.50,53102.83,47069.84
4,30.09.2019,14505071.44,0.00,0.00,0.00,0.00,1992242.84,3930445.60,1394620.78,1227905.58,939424.54,802871.19,628429.48,589692.85,457299.31,323764.87,288152.28,239872.99,192246.98,171550.69,142575.97,116853.05
5,31.10.2019,15652952.20,0.00,0.00,0.00,0.00,0.00,2289453.76,4682354.31,1659503.89,1165897.09,978861.35,763523.36,742787.97,558085.95,461806.22,358671.23,281881.11,241719.91,182730.05,144953.58,119260.10
6,30.11.2019,15107713.30,0.00,0.00,0.00,0.00,0.00,0.00,2162283.09,4637701.69,1576348.23,1144559.96,930720.35,697500.94,667277.73,547749.09,387987.02,309448.86,283876.04,215635.85,185516.45,141560.57
7,31.12.2019,17004745.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2402403.37,4947764.21,1523145.18,1245452.39,1116505.94,803590.21,724956.28,545397.33,458832.95,393971.01,333818.53,286831.44,216447.57
8,31.01.2020,16794379.95,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2502066.86,4696910.48,1505493.21,1182983.95,955821.48,846061.73,683116.81,560572.44,468203.76,349067.92,309854.99,267813.78
9,29.02.2020,19217205.82,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2833811.35,6142911.08,1833677.81,1317065.75,1108494.37,918465.33,719913.69,587381.96,498801.31,371563.77,294941.22


In [1619]:
df.shape

(20, 22)

In [1620]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date                20 non-null     object 
 1   Origination Amount  20 non-null     float64
 2   31.05.2019          20 non-null     float64
 3   30.06.2019          20 non-null     float64
 4   31.07.2019          20 non-null     float64
 5   31.08.2019          20 non-null     float64
 6   30.09.2019          20 non-null     float64
 7   31.10.2019          20 non-null     float64
 8   30.11.2019          20 non-null     float64
 9   31.12.2019          20 non-null     float64
 10  31.01.2020          20 non-null     float64
 11  29.02.2020          20 non-null     float64
 12  31.03.2020          20 non-null     float64
 13  30.04.2020          20 non-null     float64
 14  31.05.2020          20 non-null     float64
 15  30.06.2020          20 non-null     float64
 16  31.07.2020

#### Historical percentage repayement of the original amount

In [1621]:
### create a dataframe only for historical percentage repayment share of the origination amount 
df_hist = pd.DataFrame()

In [1622]:
def perc_repayment(df_hist):
    """ 
    This function compute the historical repayment percentages, 
    i.e. every repayment share of the origination amount. NOT the percentage change
    """
    for i in range(2,df.shape[1]):
        df_hist[f'{i-1}']= df.iloc[:,i]/round(df.iloc[:,1],0)


In [1623]:
perc_repayment(df_hist)
1328138.75/10018746.17	

0.1325653657118254

In [1624]:
df_hist.head(100)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,0.144037,0.332597,0.132565,0.092635,0.073504,0.053839,0.042676,0.032385,0.023661,0.016805,0.011647,0.009253,0.006328,0.005317,0.003705,0.002973,0.002448,0.001805,0.001655,0.001142
1,0.000000,0.128147,0.277124,0.113896,0.089335,0.082105,0.061533,0.046521,0.038607,0.030295,0.023483,0.018295,0.014905,0.012740,0.008497,0.007328,0.005839,0.004819,0.003991,0.003442
2,0.000000,0.000000,0.143251,0.275140,0.112570,0.081925,0.066240,0.061324,0.046904,0.039412,0.028189,0.024097,0.017868,0.015840,0.011885,0.010276,0.008363,0.006032,0.005721,0.004687
3,0.000000,0.000000,0.000000,0.128809,0.325034,0.110479,0.099343,0.070572,0.055288,0.045468,0.033222,0.026809,0.020190,0.015930,0.012092,0.008757,0.007184,0.005626,0.004228,0.003748
4,0.000000,0.000000,0.000000,0.000000,0.137348,0.270970,0.096147,0.084654,0.064765,0.055351,0.043325,0.040654,0.031527,0.022321,0.019866,0.016537,0.013254,0.011827,0.009829,0.008056
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.146263,0.299136,0.106019,0.074484,0.062535,0.048778,0.047454,0.035654,0.029503,0.022914,0.018008,0.015442,0.011674,0.009260,0.007619
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.143124,0.306976,0.104341,0.075760,0.061606,0.046169,0.044168,0.036256,0.025681,0.020483,0.018790,0.014273,0.012280,0.009370
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.141278,0.290964,0.089572,0.073241,0.065658,0.047257,0.042633,0.032073,0.026983,0.023168,0.019631,0.016868,0.012729
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.148982,0.279672,0.089643,0.070439,0.056913,0.050378,0.040675,0.033379,0.027879,0.020785,0.018450,0.015947
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.147462,0.319657,0.095419,0.068536,0.057682,0.047794,0.037462,0.030565,0.025956,0.019335,0.015348


In [1625]:
df_hist.shape

(20, 20)

### Expected repayment percentages

 1. All cash flows are assumed to be paid back over a period of 30 months. Following this assumption, the last cash flow for 
the July 2020 vintage will be collected in December 2022 and thus there are 24 months that need to be forecasted (from 
January 2021 until December 2022).
 2. Let 𝑝𝑖 denote the expected repayment percentage as a share of the origination amount. For the portfolio valuation, assume 
that the expected repayment percentages are calculated as follows:
 For the origination month, 𝑖 = 1, the value is already known and fed in. The same holds true for the following month, i = 2, except for the last vintage, December 2020, where 𝑝2 has not yet been observed. For this vintage, assume that 
the expected repayment percentage for 𝑖 = 2 will be twice the repayment of the first period (𝑝2 = 2𝑝1.for the 
December 2020 vintage).



 From the third month until the forecast horizon, 3 ≤ 𝑖 ≤ 30:

$$
p_i = \max \left\{ p_2 * \ln \left( 1 + \left( 1 - \left( 1 - \sum_{j=1}^{i-1} p_j \right) \right)*(1- \frac{i-1}{30}) \right); 0 \right\}
$$

 3. Assume that the forecasted cash flows are discounted with an annual rate of 2.5%.

In [1627]:
def forecast(df):
    len_cols = 20 #df.shape[1]
    for col in range(len_cols+1,len_cols+30): # + 30 not 31 because already have the first payment 
        df[col]=0
        for line in range(0,19): # range 19 not 20, because the last repayment we do it after
            if (col-line-1)/30 <1:
                p2 = df.iloc[line,line+1]
                ln = np.log(1+(1-np.sum(df.iloc[line,:col])))*(1-(col-line-1)/30)
                df.loc[line,col] = np.max([p2*ln,0])

            
       
    return df
    

In [1628]:
forecast_pay = forecast(df_hist)
forecast_pay.head(100)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,0.144037,0.332597,0.132565,0.092635,0.073504,0.053839,0.042676,0.032385,0.023661,0.016805,0.011647,0.009253,0.006328,0.005317,0.003705,0.002973,0.002448,0.001805,0.001655,0.001142,0.000996,0.000798,0.000639,0.000510,0.000403,0.000314,0.000237,0.000170,0.000110,0.000054,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
1,0.000000,0.128147,0.277124,0.113896,0.089335,0.082105,0.061533,0.046521,0.038607,0.030295,0.023483,0.018295,0.014905,0.012740,0.008497,0.007328,0.005839,0.004819,0.003991,0.003442,0.002914,0.002387,0.001955,0.001597,0.001296,0.001040,0.000820,0.000626,0.000452,0.000293,0.000144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2,0.000000,0.000000,0.143251,0.275140,0.112570,0.081925,0.066240,0.061324,0.046904,0.039412,0.028189,0.024097,0.017868,0.015840,0.011885,0.010276,0.008363,0.006032,0.005721,0.004687,0.004346,0.003561,0.002922,0.002396,0.001958,0.001591,0.001278,0.001008,0.000770,0.000557,0.000361,0.000177,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
3,0.000000,0.000000,0.000000,0.128809,0.325034,0.110479,0.099343,0.070572,0.055288,0.045468,0.033222,0.026809,0.020190,0.015930,0.012092,0.008757,0.007184,0.005626,0.004228,0.003748,0.003783,0.003012,0.002410,0.001934,0.001556,0.001250,0.001000,0.000793,0.000618,0.000468,0.000336,0.000217,0.000106,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
4,0.000000,0.000000,0.000000,0.000000,0.137348,0.270970,0.096147,0.084654,0.064765,0.055351,0.043325,0.040654,0.031527,0.022321,0.019866,0.016537,0.013254,0.011827,0.009829,0.008056,0.008977,0.007350,0.006033,0.004962,0.004084,0.003357,0.002751,0.002240,0.001802,0.001423,0.001089,0.000788,0.000511,0.000251,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.146263,0.299136,0.106019,0.074484,0.062535,0.048778,0.047454,0.035654,0.029503,0.022914,0.018008,0.015442,0.011674,0.009260,0.007619,0.009455,0.007580,0.006105,0.004936,0.004004,0.003254,0.002646,0.002147,0.001732,0.001384,0.001086,0.000826,0.000595,0.000385,0.000189,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.143124,0.306976,0.104341,0.075760,0.061606,0.046169,0.044168,0.036256,0.025681,0.020483,0.018790,0.014273,0.012280,0.009370,0.012710,0.010100,0.008065,0.006471,0.005214,0.004216,0.003416,0.002770,0.002242,0.001805,0.001439,0.001127,0.000857,0.000617,0.000399,0.000195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.141278,0.290964,0.089572,0.073241,0.065658,0.047257,0.042633,0.032073,0.026983,0.023168,0.019631,0.016868,0.012729,0.018383,0.014729,0.011846,0.009566,0.007753,0.006305,0.005141,0.004197,0.003426,0.002789,0.002258,0.001808,0.001421,0.001084,0.000782,0.000507,0.000248,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.148982,0.279672,0.089643,0.070439,0.056913,0.050378,0.040675,0.033379,0.027879,0.020785,0.018450,0.015947,0.022994,0.018506,0.014941,0.012104,0.009841,0.008027,0.006566,0.005382,0.004415,0.003620,0.002958,0.002403,0.001930,0.001521,0.001162,0.000840,0.000545,0.000267,0.000000,0.000000,0.000000,0.

For the last payment 

In [1629]:
def last_pay(df):
    for col in range(21,21+29): #21 + 29 not 30, because already have the first payment 
        p1 = df.loc[19,'20']
        p2 = 2*p1
        df.iloc[19,20]=p2
        ln = np.log(1+(1-np.sum(df.loc[19,:])))*np.max([(1-(col-19-1)/30),0])
        df.loc[19,col] = np.max([p2*ln,0]) 
    return df
        
        
    

In [1630]:
all_perc_repayment = last_pay(forecast_pay)
all_perc_repayment

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,0.144037,0.332597,0.132565,0.092635,0.073504,0.053839,0.042676,0.032385,0.023661,0.016805,0.011647,0.009253,0.006328,0.005317,0.003705,0.002973,0.002448,0.001805,0.001655,0.001142,0.000996,0.000798,0.000639,0.000510,0.000403,0.000314,0.000237,0.000170,0.000110,0.000054,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.128147,0.277124,0.113896,0.089335,0.082105,0.061533,0.046521,0.038607,0.030295,0.023483,0.018295,0.014905,0.012740,0.008497,0.007328,0.005839,0.004819,0.003991,0.003442,0.002914,0.002387,0.001955,0.001597,0.001296,0.001040,0.000820,0.000626,0.000452,0.000293,0.000144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.143251,0.275140,0.112570,0.081925,0.066240,0.061324,0.046904,0.039412,0.028189,0.024097,0.017868,0.015840,0.011885,0.010276,0.008363,0.006032,0.005721,0.004687,0.004346,0.003561,0.002922,0.002396,0.001958,0.001591,0.001278,0.001008,0.000770,0.000557,0.000361,0.000177,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.128809,0.325034,0.110479,0.099343,0.070572,0.055288,0.045468,0.033222,0.026809,0.020190,0.015930,0.012092,0.008757,0.007184,0.005626,0.004228,0.003748,0.003783,0.003012,0.002410,0.001934,0.001556,0.001250,0.001000,0.000793,0.000618,0.000468,0.000336,0.000217,0.000106,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.137348,0.270970,0.096147,0.084654,0.064765,0.055351,0.043325,0.040654,0.031527,0.022321,0.019866,0.016537,0.013254,0.011827,0.009829,0.008056,0.008977,0.007350,0.006033,0.004962,0.004084,0.003357,0.002751,0.002240,0.001802,0.001423,0.001089,0.000788,0.000511,0.000251,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.146263,0.299136,0.106019,0.074484,0.062535,0.048778,0.047454,0.035654,0.029503,0.022914,0.018008,0.015442,0.011674,0.009260,0.007619,0.009455,0.007580,0.006105,0.004936,0.004004,0.003254,0.002646,0.002147,0.001732,0.001384,0.001086,0.000826,0.000595,0.000385,0.000189,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.143124,0.306976,0.104341,0.075760,0.061606,0.046169,0.044168,0.036256,0.025681,0.020483,0.018790,0.014273,0.012280,0.009370,0.012710,0.010100,0.008065,0.006471,0.005214,0.004216,0.003416,0.002770,0.002242,0.001805,0.001439,0.001127,0.000857,0.000617,0.000399,0.000195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.141278,0.290964,0.089572,0.073241,0.065658,0.047257,0.042633,0.032073,0.026983,0.023168,0.019631,0.016868,0.012729,0.018383,0.014729,0.011846,0.009566,0.007753,0.006305,0.005141,0.004197,0.003426,0.002789,0.002258,0.001808,0.001421,0.001084,0.000782,0.000507,0.000248,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.148982,0.279672,0.089643,0.070439,0.056913,0.050378,0.040675,0.033379,0.027879,0.020785,0.018450,0.015947,0.022994,0.018506,0.014941,0.012104,0.009841,0.008027,0.006566,0.005382,0.004415,0.003620,0.002958,0.002403,0.001930,0.001521,0.001162,

 #### Compute the forecasted cash flows using the origination amounts.

In [1611]:
initial_amount = round(df['Origination Amount'],0)
print(initial_amount)

0     10018746.0
1     10868379.0
2     10733933.0
3     12558727.0
4     14505071.0
5     15652952.0
6     15107713.0
7     17004745.0
8     16794380.0
9     19217206.0
10    21628095.0
11    22334729.0
12    22393345.0
13    24671655.0
14    25478232.0
15    25878821.0
16    25887161.0
17    27699586.0
18    29872890.0
19    30482979.0
Name: Origination Amount, dtype: float64


In [1631]:
all_perc_repayment = all_perc_repayment.multiply(initial_amount, axis=0)
all_perc_repayment.head(100)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,1443069.08,3332200.33,1328138.75,928085.74,736418.27,539403.31,427557.86,324459.32,237056.39,168364.60,116684.68,92699.67,63399.66,53265.12,37121.13,29787.10,24524.90,18085.94,16581.01,11442.97,9.977316e+03,7.992473e+03,6.400915e+03,5.107461e+03,4.040218e+03,3.144197e+03,2.376690e+03,1703.882039,1098.330519,537.047956,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.00,1392751.60,3011884.91,1237868.70,970929.28,892351.83,668767.02,505612.59,419598.74,329262.69,255222.42,198833.96,161996.73,138461.91,92346.68,79641.30,63457.44,52373.85,43374.70,37404.87,3.167493e+04,2.594813e+04,2.124866e+04,1.735246e+04,1.408448e+04,1.130678e+04,8.909539e+03,6804.086393,4917.618219,3189.011030,1565.507147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.00,0.00,1537650.24,2953335.55,1208316.08,879375.19,711016.84,658251.40,503465.03,423045.23,302575.54,258652.52,191798.05,170027.54,127574.33,110301.21,89766.69,64746.84,61408.92,50312.70,4.664552e+04,3.822529e+04,3.136025e+04,2.571330e+04,2.102149e+04,1.707853e+04,1.372124e+04,10819.216088,8266.903565,5977.363351,3877.435500,1903.843781,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.00,0.00,0.00,1617681.94,4082016.00,1387474.94,1247623.59,886293.35,694348.63,571024.44,417223.56,336686.08,253556.20,200066.59,151859.74,109973.00,90228.14,70661.50,53102.83,47069.84,4.750345e+04,3.782580e+04,3.026234e+04,2.429428e+04,1.953562e+04,1.569725e+04,1.256091e+04,9960.252086,7766.973553,5880.693724,4221.387950,2723.711623,1332.636575,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.00,0.00,0.00,0.00,1992242.84,3930445.60,1394620.78,1227905.58,939424.54,802871.19,628429.48,589692.85,457299.31,323764.87,288152.28,239872.99,192246.98,171550.69,142575.97,116853.05,1.302077e+05,1.066059e+05,8.751408e+04,7.197343e+04,5.923155e+04,4.869723e+04,3.990453e+04,32484.475748,26142.796163,20642.631646,15790.936146,11427.795745,7417.929895,3643.817604,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.00,0.00,0.00,0.00,0.00,2289453.76,4682354.31,1659503.89,1165897.09,978861.35,763523.36,742787.97,558085.95,461806.22,358671.23,281881.11,241719.91,182730.05,144953.58,119260.10,1.479997e+05,1.186518e+05,9.555676e+04,7.726657e+04,6.267605e+04,5.094105e+04,4.141528e+04,33601.757233,27116.077280,21658.504708,16992.922162,12930.825563,9319.088019,6030.489687,2956.253080,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.00,0.00,0.00,0.00,0.00,0.00,2162283.09,4637701.69,1576348.23,1144559.96,930720.35,697500.94,667277.73,547749.09,387987.02,309448.86,283876.04,215635.85,185516.45,141560.57,1.920150e+05,1.525818e+05,1.218461e+05,9.776312e+04,7.877312e+04,6.368948e+04,5.160928e+04,41843.954415,33866.281356,27270.181084,21740.410386,17029.715482,12941.557260,9316.958322,6024.369327,2951.717843,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2402403.37,4947764.21,1523145.18,1245452.39,1116505.94,803590.21,724956.28,545397.33,458832.95,393971.01,333818.53,286831.44,216447.57,3.125949e+05,2.504553e+05,2.014405e+05,1.626592e+05,1.318412e+05,1.072175e+05,8.741472e+04,71368.729894,58253.952548,47428.128091,38389.148402,30741.714772,24171.641612,18426.029697,13297.884126,8614.054034,4225.6

#### Compute the forecasted portfolio

What is the value of the portfolio at 31.12.2020 ?
With the assumption that the forecasted cash flows are discounted with an annual rate of 2.5%.

The rate is compute as:
$$
(1+ rate_{month})^{12} = (1+rate)
$$

$$
rate_{month} = (1+rate)^{1/12}-1
$$

In [1632]:
rate_mounth = (1 + 0.025)**(1/12)-1
rate = [1/((1+rate_mounth)**i) for i in range(1,30)]
#rate - ok correct

In [1633]:
def value_discount(all_perc_repayment,rate):
    forecast_value = all_perc_repayment.iloc[:,20:] # it start with 0 so column 21 is 20 
    forecast_discount = forecast_value.multiply(rate,axis=1)
    value = np.sum(np.sum(forecast_discount))

    return value 

In [1634]:
value = value_discount(all_perc_repayment,rate)
value = round(value,2)
value

113282975.0

In [1635]:
CLIENT_VALUE = 84993122.67
relative_diff = round((value-CLIENT_VALUE)/CLIENT_VALUE,2)
relative_diff

0.33

In [1636]:
absolute_diff = round(value - CLIENT_VALUE,2)
absolute_diff

28289852.33

In [1637]:
acceptable_treshold= 500000
absolute_diff < acceptable_treshold

False

## Part II

In this part II, the forecast is computed for all repayment periods (including those already paid) and then only the forecasted periods are used (This is exactly as in the solution)

In [1657]:
df2 = pd.read_csv(r"C:\Users\lhoan\OneDrive\Bureau\Pwc - Forage\Data (2).csv",sep=';')
df2 = df.rename(columns={'Unnamed: 0': 'date'})

In [1658]:
df2.head(100)

,date,Origination Amount,31.05.2019,30.06.2019,31.07.2019,31.08.2019,30.09.2019,31.10.2019,30.11.2019,31.12.2019,31.01.2020,29.02.2020,31.03.2020,30.04.2020,31.05.2020,30.06.2020,31.07.2020,31.08.2020,30.09.2020,31.10.2020,30.11.2020,31.12.2020
0,31.05.2019,10018746.17,1443069.08,3332200.33,1328138.75,928085.74,736418.27,539403.31,427557.86,324459.32,237056.39,168364.60,116684.68,92699.67,63399.66,53265.12,37121.13,29787.10,24524.90,18085.94,16581.01,11442.97
1,30.06.2019,10868379.04,0.00,1392751.60,3011884.91,1237868.70,970929.28,892351.83,668767.02,505612.59,419598.74,329262.69,255222.42,198833.96,161996.73,138461.91,92346.68,79641.30,63457.44,52373.85,43374.70,37404.87
2,31.07.2019,10733932.61,0.00,0.00,1537650.24,2953335.55,1208316.08,879375.19,711016.84,658251.40,503465.03,423045.23,302575.54,258652.52,191798.05,170027.54,127574.33,110301.21,89766.69,64746.84,61408.92,50312.70
3,31.08.2019,12558727.02,0.00,0.00,0.00,1617681.94,4082016.00,1387474.94,1247623.59,886293.35,694348.63,571024.44,417223.56,336686.08,253556.20,200066.59,151859.74,109973.00,90228.14,70661.50,53102.83,47069.84
4,30.09.2019,14505071.44,0.00,0.00,0.00,0.00,1992242.84,3930445.60,1394620.78,1227905.58,939424.54,802871.19,628429.48,589692.85,457299.31,323764.87,288152.28,239872.99,192246.98,171550.69,142575.97,116853.05
5,31.10.2019,15652952.20,0.00,0.00,0.00,0.00,0.00,2289453.76,4682354.31,1659503.89,1165897.09,978861.35,763523.36,742787.97,558085.95,461806.22,358671.23,281881.11,241719.91,182730.05,144953.58,119260.10
6,30.11.2019,15107713.30,0.00,0.00,0.00,0.00,0.00,0.00,2162283.09,4637701.69,1576348.23,1144559.96,930720.35,697500.94,667277.73,547749.09,387987.02,309448.86,283876.04,215635.85,185516.45,141560.57
7,31.12.2019,17004745.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2402403.37,4947764.21,1523145.18,1245452.39,1116505.94,803590.21,724956.28,545397.33,458832.95,393971.01,333818.53,286831.44,216447.57
8,31.01.2020,16794379.95,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2502066.86,4696910.48,1505493.21,1182983.95,955821.48,846061.73,683116.81,560572.44,468203.76,349067.92,309854.99,267813.78
9,29.02.2020,19217205.82,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2833811.35,6142911.08,1833677.81,1317065.75,1108494.37,918465.33,719913.69,587381.96,498801.31,371563.77,294941.22


In [1659]:
def first_and_second(df):
    first_value=[]
    second_value=[]
    for i in range(19): #without the last column, I will handle it seperatly 
        first_value.append(df.iloc[i,i+2])
        second_value.append(df.iloc[i,i+2+1])

      # for the last values
    first_value.append(df.iloc[19,21])
    second_value.append(first_value[-1]*2)

    return first_value, second_value

In [1660]:
first_value, second_value = first_and_second(df2)
values = {1:first_value,2:second_value}
values
all_value = pd.DataFrame(values)
all_value # as we can see the repayment with the index 19 is not there, I will handle it separately

,1,2
0,1443069.08,3332200.33
1,1392751.60,3011884.91
2,1537650.24,2953335.55
3,1617681.94,4082016.00
4,1992242.84,3930445.60
5,2289453.76,4682354.31
6,2162283.09,4637701.69
7,2402403.37,4947764.21
8,2502066.86,4696910.48
9,2833811.35,6142911.08


In [1661]:
original_amount = df2.iloc[:,1]
percentage = all_value.divide(original_amount,axis=0)
percentage


,1,2
0,0.144037,0.332597
1,0.128147,0.277124
2,0.143251,0.275140
3,0.128809,0.325034
4,0.137348,0.270970
5,0.146263,0.299136
6,0.143124,0.306976
7,0.141278,0.290964
8,0.148982,0.279672
9,0.147462,0.319657


In [1662]:
def forecast2(df):
    for col in range(3,31): # + 30 not 31 because already have the first payment 
        df[col]=0
        for line in range(0,20): # range 19 not 20, because the last repayment we do it after
            p2 = df.iloc[line,1]
            ln = np.log(1+(1-np.sum(df.iloc[line,:col])))*(1-(col-1)/30)
            df.loc[line,col] = np.max([p2*ln,0])

       
    return df

In [1663]:
all_forecast = forecast2(percentage)
all_forecast

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
0,0.144037,0.332597,0.130664,0.099154,0.074192,0.054968,0.040496,0.029782,0.021937,0.016224,0.012071,0.009046,0.006833,0.005205,0.003998,0.003097,0.002418,0.001902,0.001505,0.001198,0.000958,0.000767,0.000614,0.000490,0.000388,0.000302,0.000228,0.000164,0.000105,0.000052
1,0.128147,0.277124,0.120712,0.096769,0.076876,0.060655,0.047639,0.037329,0.029240,0.022936,0.018040,0.014242,0.011295,0.009001,0.007210,0.005805,0.004696,0.003815,0.003111,0.002544,0.002083,0.001706,0.001393,0.001130,0.000907,0.000715,0.000546,0.000394,0.000256,0.000126
2,0.143251,0.275140,0.117727,0.094368,0.074983,0.059189,0.046520,0.036485,0.028609,0.022467,0.017692,0.013986,0.011105,0.008861,0.007106,0.005727,0.004638,0.003772,0.003079,0.002520,0.002065,0.001692,0.001382,0.001122,0.000901,0.000710,0.000543,0.000392,0.000254,0.000125
3,0.128809,0.325034,0.132198,0.101331,0.076630,0.057390,0.042731,0.031748,0.023611,0.017619,0.013218,0.009981,0.007592,0.005821,0.004498,0.003503,0.002748,0.002171,0.001726,0.001379,0.001106,0.000889,0.000713,0.000571,0.000452,0.000353,0.000267,0.000192,0.000124,0.000060
4,0.137348,0.270970,0.117548,0.094640,0.075552,0.059928,0.047332,0.037301,0.029387,0.023181,0.018333,0.014550,0.011596,0.009285,0.007470,0.006039,0.004904,0.003998,0.003271,0.002683,0.002203,0.001807,0.001479,0.001202,0.000967,0.000763,0.000583,0.000422,0.000274,0.000134
5,0.146263,0.299136,0.123185,0.096560,0.074878,0.057607,0.044103,0.033695,0.025754,0.019732,0.015179,0.011736,0.009128,0.007144,0.005627,0.004461,0.003558,0.002853,0.002299,0.001860,0.001508,0.001224,0.000992,0.000799,0.000638,0.000500,0.000380,0.000274,0.000177,0.000087
6,0.143124,0.306976,0.125546,0.097725,0.075195,0.057376,0.043555,0.032995,0.025010,0.019010,0.014512,0.011140,0.008606,0.006693,0.005242,0.004133,0.003280,0.002619,0.002101,0.001694,0.001369,0.001107,0.000895,0.000719,0.000573,0.000448,0.000341,0.000245,0.000158,0.000078
7,0.141278,0.290964,0.122109,0.096513,0.075515,0.058645,0.045326,0.034956,0.026962,0.020839,0.016163,0.012594,0.009865,0.007773,0.006161,0.004912,0.003938,0.003173,0.002569,0.002086,0.001698,0.001382,0.001123,0.000908,0.000726,0.000570,0.000434,0.000313,0.000203,0.000099
8,0.148982,0.279672,0.117967,0.094110,0.074399,0.058418,0.045671,0.035631,0.027798,0.021723,0.017028,0.013402,0.010599,0.008425,0.006733,0.005409,0.004368,0.003543,0.002885,0.002356,0.001927,0.001575,0.001285,0.001042,0.000836,0.000658,0.000502,0.000363,0.000235,0.000115
9,0.147462,0.319657,0.127438,0.097916,0.074284,0.055846,0.041761,0.031171,0.023292,0.017464,0.013162,0.009983,0.007626,0.005870,0.004553,0.003559,0.002801,0.002220,0.001769,0.001417,0.001139,0.000917,0.000738,0.000591,0.000469,0.000366,0.000277,0.000199,0.000129,0.000063


In [1647]:
value_forecast = all_forecast.multiply(original_amount,axis=0)
value_forecast

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
0,1443069.08,3332200.33,1.309092e+06,9.934001e+05,7.433153e+05,5.507089e+05,4.057167e+05,2.983817e+05,219781.118258,162548.238903,120935.069490,90624.794304,68454.839407,52142.941342,40055.411911,31026.495933,24224.037168,19052.323149,15082.939361,12005.974568,9595.706540,7686.471151,6155.651277,4911.633375,3885.231312,3023.534303,2285.453140,1638.457902,1056.150827,516.421268
1,1392751.60,3011884.91,1.311948e+06,1.051726e+06,8.355229e+05,6.592252e+05,5.177599e+05,4.057034e+05,317794.162425,249275.046705,196064.292044,154792.654691,122756.544352,97829.738467,78363.015669,63088.521288,51036.618222,41467.443988,33816.496869,27652.338552,22644.186862,18537.304849,15134.399759,12281.587534,9857.792236,7766.712684,5930.698044,4286.035688,2779.276664,1364.314607
2,1537650.24,2953335.55,1.263672e+06,1.012942e+06,8.048670e+05,6.353307e+05,4.993431e+05,3.916248e+05,307088.893578,241156.491191,189909.877971,150120.425462,119199.274114,95110.613763,76274.766857,61476.497992,49785.561022,40491.395195,33051.228457,27049.727429,22168.062498,18160.394019,14836.081664,12046.244751,9673.599161,7624.745699,5824.283279,4210.273604,2730.699812,1340.647583
3,1617681.94,4082016.00,1.660240e+06,1.272584e+06,9.623768e+05,7.207423e+05,5.366508e+05,3.987184e+05,296522.024365,221277.673207,165999.310520,125348.126400,95350.944053,73099.963046,56487.418442,43992.253372,34517.644870,27271.198750,21678.291734,17320.066815,13889.301522,11159.042019,8960.281081,7166.018673,5679.822148,4427.560268,3351.378365,2405.256843,1551.686445,759.126076
4,1992242.84,3930445.60,1.705047e+06,1.372760e+06,1.095890e+06,8.692549e+05,6.865475e+05,5.410533e+05,426256.165555,336244.651526,265920.095381,211047.104657,168203.448032,134678.395942,108354.251722,87591.579275,71128.060278,57994.256943,47445.932893,38910.938614,31948.157825,26216.077002,21448.851124,17438.117019,14019.166888,11060.404869,8455.261206,6115.935517,3968.491091,1948.934781
5,2289453.76,4682354.31,1.928214e+06,1.511453e+06,1.172056e+06,9.017207e+05,6.903487e+05,5.274331e+05,403128.073005,308868.911538,237595.030850,183705.114008,142873.121504,111817.702897,88078.243231,69821.558732,55685.789156,44659.339378,35989.350034,29113.570285,23610.113197,19160.572694,15522.986919,12511.997635,9984.243867,7827.552913,5952.881692,4288.246051,2774.081034,1359.621962
6,2162283.09,4637701.69,1.896716e+06,1.476395e+06,1.136031e+06,8.668241e+05,6.580231e+05,4.984849e+05,377848.459840,287192.452206,219245.098570,168304.762974,130018.618997,101121.595658,79191.004341,62439.718096,49552.327565,39560.141303,31748.013285,25585.862919,20678.739098,16730.537761,13517.653499,10869.810104,8656.060940,6774.506467,5144.682443,3701.864604,2392.744063,1172.075057
7,2402403.37,4947764.21,2.076428e+06,1.641186e+06,1.284121e+06,9.972359e+05,7.707589e+05,5.944237e+05,458487.986114,354356.627199,274839.683080,214150.092187,167755.745178,132173.555775,104759.441714,83521.035019,66962.592917,53962.231755,43677.471031,35473.852804,28871.587943,23505.919323,19097.750908,15431.880664,12340.829759,9692.774291,7382.475995,5324.398488,3447.408169,1690.612703
8,2502066.86,4696910.48,1.981175e+06,1.580522e+06,1.249477e+06,9.810993e+05,7.670089e+05,5.983972e+05,466842.627549,364828.493067,285978.877345,225084.055356,178000.821378,141496.268823,113079.974050,90848.932480,73355.472601,59500.159365,48447.644720,39561.873897,32356.830727,26459.379905,21581.342711,17498.531046,14034.981884,11051.056716,8434.401262,6093.012670,3949.849719,1938.560178
9,2833811.35,6142911.08,2.449009e+06,1.881678e+06,1.427523e+06,1.073203e+06,8.025388e+05,5.990201e+05,447598.871508,335602.002257,252932.062596,191846.125092,146556.928385,112808.658101,87500.981351,68385.324672,53832.264565,42659.009764,34003.893263,27236.064520,21890.865228,17623.661396,14176.825570,11356.049391,9013.266908,7034.262649,5329.602101,3827.918847,2470.869927,1209.263978


The rate is compute as:
$$
(1+ rate_{month})^{12} = (1+rate)
$$

$$
rate_{month} = (1+rate)^{1/12}-1
$$


In [1700]:
rate_mounth = (1 + 0.025)**(1/12)-1
rate = [1/((1+rate_mounth)**i) for i in range(1,30)]
rate

[0.9979443979338495,
 0.9958930213675533,
 0.9938458616151653,
 0.991802910008594,
 0.9897641578975663,
 0.9877295966495904,
 0.9856992176499193,
 0.9836730123015153,
 0.9816509720250117,
 0.9796330882586785,
 0.9776193524583845,
 0.9756097560975623,
 0.9736042906671715,
 0.9716029476756632,
 0.9696057186489431,
 0.967612595130337,
 0.9656235686805538,
 0.9636386308776504,
 0.9616577733169959,
 0.9596809876112357,
 0.9577082653902566,
 0.9557395983011512,
 0.9537749780081813,
 0.9518143961927451,
 0.9498578445533393,
 0.9479053148055263,
 0.9459567986818971,
 0.9440122879320373,
 0.9420717743224927]

In [1695]:
def forecast_sum(df):
    df_forecast = pd.DataFrame(np.nan, index=range(20), columns=range(29))
    for line in range(19, -1, -1):
        for col in range(30-20+line):
            df_forecast.loc[line,col]=df.iloc[line,20-line+col]
            
    return df_forecast


In [1701]:
only_forecast_value = forecast_sum(value_forecast)
only_forecast_value

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28
0,9.595707e+03,7.686471e+03,6.155651e+03,4.911633e+03,3.885231e+03,3.023534e+03,2.285453e+03,1638.457902,1056.150827,516.421268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.765234e+04,2.264419e+04,1.853730e+04,1.513440e+04,1.228159e+04,9.857792e+03,7.766713e+03,5930.698044,4286.035688,2779.276664,1364.314607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.305123e+04,2.704973e+04,2.216806e+04,1.816039e+04,1.483608e+04,1.204624e+04,9.673599e+03,7624.745699,5824.283279,4210.273604,2730.699812,1340.647583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.727120e+04,2.167829e+04,1.732007e+04,1.388930e+04,1.115904e+04,8.960281e+03,7.166019e+03,5679.822148,4427.560268,3351.378365,2405.256843,1551.686445,759.126076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7.112806e+04,5.799426e+04,4.744593e+04,3.891094e+04,3.194816e+04,2.621608e+04,2.144885e+04,17438.117019,14019.166888,11060.404869,8455.261206,6115.935517,3968.491091,1948.934781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6.982156e+04,5.568579e+04,4.465934e+04,3.598935e+04,2.911357e+04,2.361011e+04,1.916057e+04,15522.986919,12511.997635,9984.243867,7827.552913,5952.881692,4288.246051,2774.081034,1359.621962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7.919100e+04,6.243972e+04,4.955233e+04,3.956014e+04,3.174801e+04,2.558586e+04,2.067874e+04,16730.537761,13517.653499,10869.810104,8656.060940,6774.506467,5144.682443,3701.864604,2392.744063,1172.075057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.321736e+05,1.047594e+05,8.352104e+04,6.696259e+04,5.396223e+04,4.367747e+04,3.547385e+04,28871.587943,23505.919323,19097.750908,15431.880664,12340.829759,9692.774291,7382.475995,5324.398488,3447.408169,1690.612703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.780008e+05,1.414963e+05,1.130800e+05,9.084893e+04,7.335547e+04,5.950016e+04,4.844764e+04,39561.873897,32356.830727,26459.379905,21581.342711,17498.531046,14034.981884,11051.056716,8434.401262,6093.012670,3949.849719,1938.560178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1.918461e+05,1.465569e+05,1.128087e+05,8.750098e+04,6.838532e+04,5.383226e+04,4.265901e+04,34003.893263,27236.064520,21890.865228,17623.661396,14176.825570,11356.049391,9013.266908,7034.262649,5329.602101,3827.918847,2470.869927,1209.263978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1702]:
def value_discount(df,rate):
    value = df.multiply(rate)

    return value 

In [1703]:
value = value_discount(only_forecast_value,rate)
value

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28
0,9.575982e+03,7.654903e+03,6.117769e+03,4.871372e+03,3.845463e+03,2.986434e+03,2.252769e+03,1611.706820,1036.771486,505.903361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.759550e+04,2.255119e+04,1.842322e+04,1.501034e+04,1.215588e+04,9.736833e+03,7.655643e+03,5833.867610,4207.391099,2722.671382,1333.780363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.298329e+04,2.693863e+04,2.203164e+04,1.801153e+04,1.468422e+04,1.189843e+04,9.535259e+03,7500.256570,5717.413342,4124.523333,2669.584982,1307.948862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.721514e+04,2.158926e+04,1.721348e+04,1.377545e+04,1.104482e+04,8.850335e+03,7.063539e+03,5587.087762,4346.318841,3283.121138,2351.425637,1513.840434,739.088404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7.098185e+04,5.775608e+04,4.715394e+04,3.859198e+04,3.162114e+04,2.589440e+04,2.114212e+04,17153.405097,13761.928803,10835.138580,8266.026985,5966.766358,3863.739954,1893.590778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6.967803e+04,5.545709e+04,4.438450e+04,3.569434e+04,2.881557e+04,2.332041e+04,1.888656e+04,15269.543302,12282.414640,9780.895654,7652.367210,5807.689455,4175.054755,2695.305309,1318.297230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7.902822e+04,6.218328e+04,4.924738e+04,3.923586e+04,3.142305e+04,2.527191e+04,2.038302e+04,16457.378477,13269.617697,10648.425641,8462.332691,6609.274602,5008.884900,3596.742561,2320.018327,1134.114588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.319019e+05,1.043292e+05,8.300704e+04,6.641369e+04,5.340988e+04,4.314153e+04,3.496655e+04,28400.201882,23074.608552,18708.788701,15086.505182,12039.833911,9436.926639,7172.835438,5162.567222,3335.755565,1632.495472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.776349e+05,1.409151e+05,1.123841e+05,9.010424e+04,7.260462e+04,5.877007e+04,4.775481e+04,38915.947669,31763.114335,25920.484050,21098.338287,17071.737606,13664.518582,10737.239281,8178.043697,5895.675802,3814.067982,1868.071476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1.914518e+05,1.459550e+05,1.121144e+05,8.678373e+04,6.768534e+04,5.317172e+04,4.204895e+04,33448.712116,26736.309211,21445.015908,17229.232442,13831.049337,11056.298412,8757.316696,6820.461291,5156.990120,3696.328657,2381.025713,1162.898105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1710]:
portfolio_value = np.sum(np.sum(value))
portfolio_value

84779941.82276757

In [1714]:
# compute with the client forecast
CLIENT_FORECAST = 84993122.67
asboslute_diff = CLIENT_FORECAST - portfolio_value
asboslute_diff 

213180.84723243117

In [1715]:
relative_diff = (CLIENT_FORECAST - portfolio_value)/portfolio_value
relative_diff

0.0025145198575163646

In [1717]:
asboslute_diff < 500000

True

So yes, our portfolio valuation fits with the one with the client (difference less than 500'000)